In [1]:
!pip install pymilvus

   ---------------------------------------- 0.0/4.8 MB ? eta -:--:--
   ---------------------------------------  4.7/4.8 MB 25.1 MB/s eta 0:00:01
   ---------------------------------------- 4.8/4.8 MB 22.2 MB/s  0:00:00

   ------------- -------------------------- 1/3 [grpcio]
   ------------- -------------------------- 1/3 [grpcio]
   ------------- -------------------------- 1/3 [grpcio]
   ------------- -------------------------- 1/3 [grpcio]
   ------------- -------------------------- 1/3 [grpcio]
   -------------------------- ------------- 2/3 [pymilvus]
   -------------------------- ------------- 2/3 [pymilvus]
   -------------------------- ------------- 2/3 [pymilvus]
   -------------------------- ------------- 2/3 [pymilvus]
   -------------------------- ------------- 2/3 [pymilvus]
   -------------------------- ------------- 2/3 [pymilvus]
   -------------------------- ------------- 2/3 [pymilvus]
   -------------------------- ------------- 2/3 [pymilvus]
   -------------------

In [2]:
from pymilvus import connections, utility

# Connect to Milvus
connections.connect(
    alias="default",
    host="localhost",
    port="19530"
)

print("✅ Connected to Milvus")

# Optional: Check existing collections
collections = utility.list_collections()
print("Existing collections:", collections)


✅ Connected to Milvus
Existing collections: []
